In [117]:
import tensorflow as tf
import os
import matplotlib.pyplot as plt
%matplotlib inline

from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [125]:
import pandas as pd
import numpy as np
import os
import keras
import matplotlib.pyplot as plt
from keras.layers import Dense,GlobalAveragePooling2D , Flatten
from keras.applications import MobileNet
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam , RMSprop
from keras.applications.inception_v3 import preprocess_input
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, TensorBoard

In [3]:
from keras.applications import VGG16
from keras.models import load_model
from keras.applications import MobileNetV2

import cv2
import numpy as np

In [4]:
base_model = MobileNetV2(weights='imagenet',include_top=False,input_shape = (128,128,3))

In [5]:
base_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu

In [6]:
#log weight
base_model.trainable=False

In [126]:
x = (base_model.output)
x = Dense(1024,activation = 'relu')(x)
x = Dense(1024,activation = 'relu')(x)
x = Dense(512,activation = 'relu')(x)
prediction_layer = Flatten()(x)
prediction_layer = Dense(units=20,activation='softmax')(prediction_layer)

In [36]:
model=Model(inputs=base_model.input,outputs=prediction_layer)

In [37]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu

In [28]:
dirs = "C:/Users/SMILE/DataSet/Train_data/"
dirs_val = "C:/Users/SMILE/DataSet/Validation_data/"

In [156]:
data_gen_train = ImageDataGenerator(preprocessing_function = preprocess_input)
data_gen_val = ImageDataGenerator(preprocessing_function = preprocess_input)

In [157]:
train_generator = data_gen_train.flow_from_directory(dirs,target_size = (128,128),batch_size = 30 ,
                                                     class_mode='categorical',shuffle=True)
valid_generator = data_gen_val.flow_from_directory(dirs_val , target_size=(128,128),batch_size = 30 , class_mode = 'categorical',shuffle=True)

Found 5583 images belonging to 20 classes.
Found 868 images belonging to 20 classes.


In [31]:
file_path = 'C:/Users/SMILE/DataSet/weight_b/Best_Weight.h5'

In [32]:
checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
early = EarlyStopping(monitor="val_acc", mode="max", patience=15)
callbacks_list = [checkpoint, early] #early

In [39]:
model.compile(optimizer= Adam(lr=0.0001),loss='categorical_crossentropy',metrics=['accuracy'])
# Adam optimizer
# loss function will be categorical cross entropy
# evaluation metric will be accuracy

In [40]:
step_size_train=train_generator.n//train_generator.batch_size
step_size_validation=valid_generator.n//valid_generator.batch_size
model.fit_generator(train_generator,epochs=25 , callbacks=callbacks_list , steps_per_epoch = step_size_train
                   , validation_data = valid_generator , validation_steps = step_size_validation)

Epoch 1/25
186/186 [==============================] - 54s 292ms/step - loss: 0.6224 - acc: 0.8339 - val_loss: 0.3404 - val_acc: 0.9153

Epoch 00001: val_acc improved from -inf to 0.91527, saving model to C:/Users/SMILE/DataSet/weight_b/Best_Weight.h5
Epoch 2/25
186/186 [==============================] - 34s 180ms/step - loss: 0.0443 - acc: 0.9864 - val_loss: 0.2694 - val_acc: 0.9236

Epoch 00002: val_acc improved from 0.91527 to 0.92363, saving model to C:/Users/SMILE/DataSet/weight_b/Best_Weight.h5
Epoch 3/25
186/186 [==============================] - 34s 180ms/step - loss: 0.0184 - acc: 0.9953 - val_loss: 0.4309 - val_acc: 0.9024

Epoch 00003: val_acc did not improve from 0.92363
Epoch 4/25
186/186 [==============================] - 33s 180ms/step - loss: 0.0129 - acc: 0.9968 - val_loss: 0.2864 - val_acc: 0.9296

Epoch 00004: val_acc improved from 0.92363 to 0.92959, saving model to C:/Users/SMILE/DataSet/weight_b/Best_Weight.h5
Epoch 5/25
186/186 [==============================] - 3

In [163]:
#jun model
model.load_weights(file_path2)

In [172]:
test_idg = ImageDataGenerator(preprocessing_function=preprocess_input)
dirs3 = "C:/Users/SMILE/DataSet/"

In [191]:
test_gen = test_idg.flow_from_directory(dirs3,
    target_size=(128, 128),
    batch_size = 20,
    shuffle = False,
    class_mode='binary'
    ,classes = ['Test_data']

)

Found 102 images belonging to 1 classes.


In [192]:
len(test_gen.filenames)

102

In [193]:
step_size_test=test_gen.n//test_gen.batch_size
predicts = model.predict_generator(test_gen ,verbose = 1 , steps=6)

6/6 [==============================] - 0s 41ms/step


In [194]:
print(predicts[0])
print(np.argmax(predicts[0]))
label_index = {v: k for k,v in train_generator.class_indices.items()}
print(label_index)
x = np.argmax(predicts , axis = 1)
print(x)
len(predicts)


[9.9859673e-01 1.2989956e-03 7.8892867e-07 1.3714208e-07 2.0571074e-09
 6.5007956e-12 5.8671683e-12 7.7934153e-08 5.0374029e-09 2.7982782e-08
 8.6740366e-09 9.5599857e-09 5.0674959e-10 5.5023497e-07 3.4671005e-10
 2.7572817e-08 1.2912155e-08 2.6814286e-09 1.0023042e-04 2.3234543e-06]
0
{0: '0', 1: '1', 2: '10', 3: '11', 4: '12', 5: '13', 6: '14', 7: '15', 8: '16', 9: '17', 10: '18', 11: '19', 12: '2', 13: '3', 14: '4', 15: '5', 16: '6', 17: '7', 18: '8', 19: '9'}
[ 0 13  0  6  0  0  6  1  0 12 13  8  1  1  1  1 12 12  1 12  0 13 14 18
 13 13 13 18  1 13 13 13  1 18  1 18 14 14 14 14 14 14  1  1 14 14 14 14
 14 15  6  6  1 13 18 18 13  1 15 15 14 14 15  1 15  3 13  1  0  1 16 15
  1  8 17 17 17 17 14 17 14 14 17 17 17  1 18  1 18 14  1 19 13  9 19 19
  1 19 19 19 19 12]


102

In [195]:
y = np.argmax(predicts , axis = 1)
label_index = {v: k for k,v in train_generator.class_indices.items()}
y = [label_index[p] for p in y]

df = pd.DataFrame(columns=['fname', 'number'])
df['fname'] = [os.path.basename(x) for x in test_gen.filenames]
df['number'] = y
df.to_csv("C:/Users/SMILE/DataSet/test-pre_train_data.csv", index=False)

In [146]:
len(base_model.layers)

155

In [ ]:
base_model.trainable = True

In [147]:
for layer in base_model.layers[:100]:
    layer.trainable = False

In [158]:
file_path2 = 'C:/Users/SMILE/DataSet/weight_b/Best_Weight_open.h5'

In [159]:
checkpoint = ModelCheckpoint(file_path2, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
early = EarlyStopping(monitor="val_acc", mode="max", patience=15)
callbacks_list = [checkpoint, early] #early

In [160]:
model.compile(optimizer= Adam(lr=0.0001),loss='categorical_crossentropy',metrics=['accuracy'])
# Adam optimizer
# loss function will be categorical cross entropy
# evaluation metric will be accuracy

In [161]:
step_size_train=train_generator.n//train_generator.batch_size
step_size_validation=valid_generator.n//valid_generator.batch_size
model.fit_generator(train_generator,epochs=15 , callbacks=callbacks_list , steps_per_epoch = step_size_train
                   , validation_data = valid_generator , validation_steps = step_size_validation)

Epoch 1/15
186/186 [==============================] - 27s 147ms/step - loss: 0.0153 - acc: 0.9955 - val_loss: 0.3171 - val_acc: 0.9381

Epoch 00001: val_acc improved from -inf to 0.93810, saving model to C:/Users/SMILE/DataSet/weight_b/Best_Weight_open.h5
Epoch 2/15
186/186 [==============================] - 16s 86ms/step - loss: 0.0053 - acc: 0.9984 - val_loss: 0.5187 - val_acc: 0.9165

Epoch 00002: val_acc did not improve from 0.93810
Epoch 3/15
186/186 [==============================] - 16s 86ms/step - loss: 0.0177 - acc: 0.9968 - val_loss: 0.8291 - val_acc: 0.8532

Epoch 00003: val_acc did not improve from 0.93810
Epoch 4/15
186/186 [==============================] - 16s 87ms/step - loss: 0.0188 - acc: 0.9961 - val_loss: 0.3041 - val_acc: 0.9511

Epoch 00004: val_acc improved from 0.93810 to 0.95107, saving model to C:/Users/SMILE/DataSet/weight_b/Best_Weight_open.h5
Epoch 5/15
186/186 [==============================] - 16s 87ms/step - loss: 0.0290 - acc: 0.9923 - val_loss: 0.4314 

In [171]:
model.save('DataSet/Model/MobileNet2_model.h5')